<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1


    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66


    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-19 13:43:49
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  6.17 L
Current:  1.43 C
-------------------
Today PnL: -1.42 K (-0.01%)
Current PnL: -16.79 L (-11.29%)
CY Booked + Current PnL: -5.35 L (-3.6%)
-------------------
Total profit:  2.05 L
Total loss:  -18.84 L
-------------------
Total Booked + Current PnL: 21.63 L (17.78%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.01%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.24 C
Est FTT PnL: 81.30 L (56.93%)
Deployed:  1.22 C
Current:  1.43 C
CAGR/XIRR %: 9.9%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.00,13.61,65.0,H-MC,3.49,196200.0,31530.0,11792.0,0.05,19.15,6.01,26.31,89.0,2.67,1.44,28.66,X40N,NTT,AC
76,TTKPRESTIG,770.00,101.71,50.0,M-SC,3.72,86905.0,-13872.0,13966.0,-1.51,-13.77,16.07,0.09,245.0,-0.99,0.64,14.17,OX40N,NTT,DURABLES
83,VOLTAS,1530.00,1.58,59.0,H-MC,2.68,213090.0,21348.0,16408.0,-0.39,11.13,7.70,19.69,99.0,1.30,1.56,18.69,XY25,NTT,AC
49,NATIONALUM,244.55,-43.66,65.0,H-MC,7.28,111989.0,11485.0,17627.0,0.25,11.43,15.74,28.96,79.0,0.65,0.82,49.28,MH,ATH,METALS
56,RELIANCE,1533.00,-11.79,59.0,H-LC,4.96,220397.0,10031.0,18756.0,-0.16,4.77,8.51,13.68,37.0,0.53,1.61,22.08,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
70,TANLA,1963.11,-15.81,77.0,H-SC,12.38,225752.0,-45009.0,374951.0,5.45,-16.62,166.09,121.86,133.0,-0.12,1.65,76.60,AR,ATH,IT
60,SAMMAANCAP,326.00,-175.00,71.0,M-SC,3.34,86442.0,-15768.0,109159.0,3.51,-15.43,126.28,91.37,208.0,-0.14,0.63,40.32,XY25,NTT,FINANCE
78,UNIONBANK,163.00,-11.63,68.0,M-LC,7.07,160181.0,19341.0,25148.0,3.14,13.73,15.70,31.59,66.0,0.77,1.17,43.86,XY24,NTT,BANKS
59,SAIL,228.00,48.36,70.0,M-MC,7.55,233727.0,8765.0,158888.0,1.91,3.90,67.98,74.53,192.0,0.06,1.71,37.41,XY24,BTT,STEEL
11,BANKINDIA,190.00,-27.95,68.0,H-MC,6.36,186129.0,6321.0,106094.0,1.69,3.52,57.00,62.52,88.0,0.06,1.36,38.80,XR,NTT,BANKS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,QUESS,986.00,-41.60,43.0,M-SC,39.88,58035.0,-6971.0,157902.0,-2.79,-10.72,272.08,232.18,198.0,-0.04,0.42,1.07,XY24,NTT,MISC
80,VAIBHAVGBL,521.00,61.43,64.0,H-SC,6.64,140047.0,-42728.0,153800.0,-2.51,-23.38,109.82,60.77,125.0,-0.28,1.02,25.43,XR,NTT,JEWELLERY
36,INDIAMART,4850.92,-53.15,34.0,H-SC,11.48,130778.0,7442.0,126318.0,-2.20,6.03,96.59,108.45,119.0,0.06,0.96,29.74,AR,ATH,MISC
55,RELAXO,1176.00,-32.81,53.0,H-SC,1.92,91575.0,-53585.0,129515.0,-1.68,-36.91,141.43,52.31,136.0,-0.41,0.67,22.58,X40N,NTT,FOOTWEAR
15,BSOFT,836.99,-13.25,52.0,H-SC,14.75,97372.0,-43281.0,116058.0,-1.65,-30.77,119.19,51.74,131.0,-0.37,0.71,11.04,XR,ATH,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,ICICIPRULI,790.00,-20.54,46.0,H-MC,2.18,136707.0,919.0,41832.0,-0.02,0.68,30.60,31.48,107.0,0.02,1.00,12.88,X40,ATH,INSURANCE
29,HAVELLS,2069.16,0.34,59.0,H-MC,1.92,248758.0,1027.0,74030.0,0.02,0.41,29.76,30.30,92.0,0.01,1.82,13.76,X40,ATH,ELECTRICAL
33,ICICIGI,2260.25,-20.11,45.0,H-MC,2.31,137374.0,1241.0,29879.0,-1.55,0.91,21.75,22.86,91.0,0.04,1.01,13.64,X40,ATH,INSURANCE
51,PGHH,17973.08,-30.81,52.0,H-MC,4.09,200085.0,-735.0,69510.0,-0.10,-0.37,34.74,34.25,80.0,-0.01,1.46,4.81,X40,ATH,FMCG


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,96.03,56.0,H-SC,9.43,124173.0,-18306.0,37823.0,1.05,-12.85,30.46,13.70,163.0,-0.48,0.91,46.56,OX40N,NTT,BANKS
76,TTKPRESTIG,770.0,101.71,50.0,M-SC,3.72,86905.0,-13872.0,13966.0,-1.51,-13.77,16.07,0.09,245.0,-0.99,0.64,14.17,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.06,62.0,H-SC,1.26,226800.0,-42867.0,79199.0,-0.49,-15.90,34.92,13.47,138.0,-0.54,1.66,14.71,XY24,NTT,PAINTS
18,CERA,9475.0,-20.29,45.0,H-SC,2.10,145452.0,-30451.0,72479.0,-0.37,-17.31,49.83,23.89,149.0,-0.42,1.06,25.62,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-19.68,54.0,H-MC,7.44,108680.0,-23155.0,65382.0,-0.24,-17.56,60.16,32.03,98.0,-0.35,0.80,21.73,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,48.36,70.0,M-MC,7.55,233727.0,8765.0,158888.0,1.91,3.90,67.98,74.53,192.0,0.06,1.71,37.41,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,13.61,65.0,H-MC,3.49,196200.0,31530.0,11792.0,0.05,19.15,6.01,26.31,89.0,2.67,1.44,28.66,X40N,NTT,AC
83,VOLTAS,1530.00,1.58,59.0,H-MC,2.68,213090.0,21348.0,16408.0,-0.39,11.13,7.70,19.69,99.0,1.30,1.56,18.69,XY25,NTT,AC
17,CAMS,5211.76,0.38,67.0,H-SC,3.81,114106.0,12102.0,31824.0,1.53,11.86,27.89,43.06,122.0,0.38,0.84,31.01,X40N,ATH,MISC
29,HAVELLS,2069.16,0.34,59.0,H-MC,1.92,248758.0,1027.0,74030.0,0.02,0.41,29.76,30.30,92.0,0.01,1.82,13.76,X40,ATH,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,244.55,-43.66,65.0,H-MC,7.28,111989.0,11485.0,17627.0,0.25,11.43,15.74,28.96,79.0,0.65,0.82,49.28,MH,ATH,METALS
84,WHIRLPOOL,2270.00,-40.79,50.0,M-SC,0.65,100042.0,8544.0,70209.0,-0.11,9.34,70.18,86.07,223.0,0.12,0.73,45.28,XR,NTT,DURABLES
85,WIPRO,420.00,-10.16,57.0,M-LC,5.66,158230.0,7285.0,102169.0,-0.67,4.83,64.57,72.51,53.0,0.07,1.16,11.14,XR,NTT,IT
11,BANKINDIA,190.00,-27.95,68.0,H-MC,6.36,186129.0,6321.0,106094.0,1.69,3.52,57.00,62.52,88.0,0.06,1.36,38.80,XR,NTT,BANKS
36,INDIAMART,4850.92,-53.15,34.0,H-SC,11.48,130778.0,7442.0,126318.0,-2.20,6.03,96.59,108.45,119.0,0.06,0.96,29.74,AR,ATH,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VBL,671.64,-15.98,43.0,H-LC,6.56,302412.0,-13430.0,126106.0,0.10,-4.25,41.70,35.67,5.0,-0.11,2.21,9.01,X40N,ATH,FMCG
5,ASIANPAINT,3465.66,-12.67,49.0,H-LC,7.80,220308.0,-31460.0,84664.0,1.00,-12.50,38.43,21.13,27.0,-0.37,1.61,18.50,X40,ATH,PAINTS
41,ITC,452.00,-37.92,49.0,H-LC,2.38,200490.0,352.0,20530.0,-0.38,0.18,10.24,10.44,4.0,0.02,1.47,5.75,X40,NTT,FMCG
30,HCLTECH,1943.91,1.08,49.0,H-LC,9.73,230994.0,-10930.0,74195.0,-1.51,-4.52,32.12,26.15,8.0,-0.15,1.69,9.16,X40,ATH,IT
79,UNITDSPR,1644.00,-13.09,51.0,H-LC,7.77,228003.0,-6141.0,54766.0,-0.14,-2.62,24.02,20.77,86.0,-0.11,1.67,4.11,X40N,NTT,BREWERIES


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.2,0.00,68.0,H-LC,1.47,247815.0,1664.0,77541.0,-0.32,0.68,31.29,32.18,16.0,0.02,1.81,36.96,X200,ATH,IT
41,ITC,452.0,-37.92,49.0,H-LC,2.38,200490.0,352.0,20530.0,-0.38,0.18,10.24,10.44,4.0,0.02,1.47,5.75,X40,NTT,FMCG
50,NESTLEIND,1377.0,-9.24,52.0,H-LC,4.18,279279.0,13853.0,42925.0,-1.29,5.22,15.37,21.40,11.0,0.32,2.04,12.93,XY25,NTT,FMCG
56,RELIANCE,1533.0,-11.79,59.0,H-LC,4.96,220397.0,10031.0,18756.0,-0.16,4.77,8.51,13.68,37.0,0.53,1.61,22.08,XY25,NTT,REFINERIES
19,CIPLA,1795.0,-19.78,52.0,H-LC,5.72,214254.0,9754.0,31667.0,-0.91,4.77,14.78,20.25,10.0,0.31,1.57,13.92,X40N,BTT,PHARMA


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UNITDSPR,1644.00,-13.09,51.0,H-LC,7.77,228003.0,-6141.0,54766.0,-0.14,-2.62,24.02,20.77,86.0,-0.11,1.67,4.11,X40N,NTT,BREWERIES
73,TCS,4476.75,-25.47,57.0,H-LC,12.30,296918.0,-49038.0,123904.0,-0.57,-14.17,41.73,21.64,1.0,-0.40,2.17,5.18,X40,ATH,IT
41,ITC,452.00,-37.92,49.0,H-LC,2.38,200490.0,352.0,20530.0,-0.38,0.18,10.24,10.44,4.0,0.02,1.47,5.75,X40,NTT,FMCG
82,VBL,671.64,-15.98,43.0,H-LC,6.56,302412.0,-13430.0,126106.0,0.10,-4.25,41.70,35.67,5.0,-0.11,2.21,9.01,X40N,ATH,FMCG
30,HCLTECH,1943.91,1.08,49.0,H-LC,9.73,230994.0,-10930.0,74195.0,-1.51,-4.52,32.12,26.15,8.0,-0.15,1.69,9.16,X40,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,7969.85,3.38,65.0,H-LC,26.99,166290.0,-19805.0,232207.0,0.24,-10.64,139.64,114.13,15.0,-0.09,1.22,22.46,AR,ATH,ELECTRICAL
41,ITC,452.00,-37.92,49.0,H-LC,2.38,200490.0,352.0,20530.0,-0.38,0.18,10.24,10.44,4.0,0.02,1.47,5.75,X40,NTT,FMCG
19,CIPLA,1795.00,-19.78,52.0,H-LC,5.72,214254.0,9754.0,31667.0,-0.91,4.77,14.78,20.25,10.0,0.31,1.57,13.92,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-12.67,49.0,H-LC,7.80,220308.0,-31460.0,84664.0,1.00,-12.50,38.43,21.13,27.0,-0.37,1.61,18.50,X40,ATH,PAINTS
56,RELIANCE,1533.00,-11.79,59.0,H-LC,4.96,220397.0,10031.0,18756.0,-0.16,4.77,8.51,13.68,37.0,0.53,1.61,22.08,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4476.75,-25.47,57.0,H-LC,12.30,296918.0,-49038.0,123904.0,-0.57,-14.17,41.73,21.64,1.0,-0.40,2.17,5.18,X40,ATH,IT
39,INFY,2275.00,-16.73,55.0,H-LC,8.19,324910.0,12022.0,159661.0,-0.99,3.84,49.14,54.87,3.0,0.08,2.38,10.75,X40,BTT,IT
41,ITC,452.00,-37.92,49.0,H-LC,2.38,200490.0,352.0,20530.0,-0.38,0.18,10.24,10.44,4.0,0.02,1.47,5.75,X40,NTT,FMCG
82,VBL,671.64,-15.98,43.0,H-LC,6.56,302412.0,-13430.0,126106.0,0.10,-4.25,41.70,35.67,5.0,-0.11,2.21,9.01,X40N,ATH,FMCG
1,ABB,7934.00,-37.53,68.0,H-LC,9.86,260885.0,-734.0,119955.0,-0.02,-0.28,45.98,45.57,7.0,-0.01,1.91,11.01,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,7033.33,66.0,L-SC,5.21,78248.0,-15562.0,92184.0,0.88,-16.59,117.81,81.67,269.0,-0.17,0.57,51.53,XR,NTT,CERAMICS
21,COFFEEDAY,80.00,-41.65,52.0,L-SC,42.28,85229.0,-28320.0,68371.0,0.98,-24.94,80.22,35.27,268.0,-0.41,0.62,108.60,XR,NTT,HOTELS
60,SAMMAANCAP,326.00,-175.00,71.0,M-SC,3.34,86442.0,-15768.0,109159.0,3.51,-15.43,126.28,91.37,208.0,-0.14,0.63,40.32,XY25,NTT,FINANCE
3,ALKYLAMINE,4546.37,-4.51,47.0,H-SC,12.49,88907.0,-12056.0,106591.0,-0.37,-11.94,119.89,93.63,148.0,-0.11,0.65,32.42,SR,ATH,CHEMICALS
48,MASFIN,398.61,-16.89,53.0,H-SC,12.99,94650.0,-3330.0,24931.0,0.45,-3.40,26.34,22.05,152.0,-0.13,0.69,37.58,XR,ATH,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SHALBY,327.00,1108.93,69.0,M-SC,4.59,164359.0,-16784.0,62259.0,0.79,-9.27,37.88,25.10,235.0,-0.27,1.20,31.72,XY24,NTT,HEALTHCARE
70,TANLA,1963.11,-15.81,77.0,H-SC,12.38,225752.0,-45009.0,374951.0,5.45,-16.62,166.09,121.86,133.0,-0.12,1.65,76.60,AR,ATH,IT
60,SAMMAANCAP,326.00,-175.00,71.0,M-SC,3.34,86442.0,-15768.0,109159.0,3.51,-15.43,126.28,91.37,208.0,-0.14,0.63,40.32,XY25,NTT,FINANCE
7,ATULAUTO,844.00,3840.54,61.0,M-SC,5.36,122755.0,-24613.0,76427.0,-1.52,-16.70,62.26,35.16,236.0,-0.32,0.90,25.38,XY24,NTT,AUTO
74,TITAGARH,1548.00,1.18,69.0,H-SC,3.16,165821.0,-26194.0,105081.0,0.11,-13.64,63.37,41.08,158.0,-0.25,1.21,40.51,XY24,NTT,ENGINEERING


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.06
1,25,43.96
2,50,74.50


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.52
LC    32.09
MC    23.34
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.37
X40      15.36
X40N     11.95
XR       11.76
AR        8.82
XY25      8.25
OX40N     8.20
X200      1.81
MH        1.70
X5K       1.44
SR        1.29
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.31
H-LC    25.36
H-MC    20.32
M-SC    13.63
M-LC     5.66
M-MC     2.71
L-SC     1.58
L-LC     1.07
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.68,-12.43,73.74
FMCG,11.88,-4.18,40.63
FINANCE,9.76,-15.26,60.41
MISC,7.43,-13.24,76.20
BANKS,6.39,-14.24,74.34
PAINTS,5.86,-15.20,32.37
ELECTRICAL,5.77,-4.56,62.19
AC,3.72,4.67,11.29
INSURANCE,3.44,-7.12,43.86


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2830231.0,22
XR,1298751.0,14
AR,1291667.0,9
X40,749290.0,10
X40N,573907.0,9
OX40N,526249.0,10
XY25,335887.0,6
SR,245163.0,2
X5K,120630.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3404054.0,29
M-SC,1380862.0,17
H-LC,1226828.0,14
H-MC,1092334.0,15
M-LC,376241.0,4
M-MC,307008.0,2
L-SC,244160.0,3
L-MC,60369.0,1
L-LC,38471.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1022575.0      6
M-SC       XY24       799312.0      7
H-SC       AR         778848.0      5
           XR         755044.0      7
H-MC       XY24       487943.0      4
H-LC       X40        462954.0      5
           AR         352162.0      2
M-MC       XY24       307008.0      2
H-SC       X40N       287907.0      4
           OX40N      251133.0      4
           SR         245163.0      2
H-MC       X40        215251.0      4
H-LC       X40N       212539.0      3
M-SC       AR         160657.0      2
L-SC       XR         160555.0      2
M-LC       XY24       153442.0      2
H-MC       XY25       126576.0      2
M-SC       OX40N      126129.0      4
M-LC       X5K        120630.0      1
M-SC       XR         114520.0      2
           XY25       109159.0      1
H-MC       XR         106094.0      1
M-LC       XR         102169.0      1
L-SC       OX40N       83605.0      1
H-LC       X200        77541.0      1
H-MC       X40N        73461.0      2
M-SC       X40         71085.0      1
H-MC       OX40N       65382.0      1
H-SC       MH          63384.0      1
H-LC       XY25        61681.0      2
L-MC       XR          60369.0      1
H-LC       XY24        59951.0      1
L-LC       XY25        38471.0      1
H-MC       MH          17627.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
